In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [3]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
import qiskit.providers.aer.noise as noise
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation, apply_checking_circuit
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.5.1', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

qubits = 5

simulation_backend = "qasm_simulator"

seed = 0
print_circuits = True

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibm_auckland')
device

<IBMQBackend('ibm_auckland') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
simulator = provider.get_backend('ibmq_qasm_simulator')

In [9]:
device.configuration().to_dict()['max_shots']

100000

In [10]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [11]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [12]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_front.qasm")

In [13]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_mid.qasm")

In [14]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_end.qasm")

In [15]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23955/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [16]:
orign_circuit.draw()

┌───┐       ┌────────────────┐┌────────────┐   ┌─────────────────┐»
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘ │ └─────────────────┘»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                           │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────────────────────────■─────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                      »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘                      »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├────────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘                                    »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──────────────────────────────────────»
     └─────────────────┘  └────────────┘                                      »
q_6: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«      ┌──────────────┐                                                     »
«q_0: ─┤ Rz(-0.23807) ├─────────────────────────────────────────────────────»
«     ┌┴──────────────┤┌──────────────┐                                     »
«q_1: ┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├─────────────────────────────────────»
«     └───────────────┘├─────────────┬┘  ┌────────────┐                     »
«q_2: ────────■────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├─────────────────────»
«             │        └─────────────┘ ┌─┴────────────┴─┐  ┌──────────────┐ »
«q_3: ────────■───────────────■────────┤ Ry(2.6997e-05) ├──┤ Rz(0.022658) ├─»
«                             │        └────────────────┘  ├──────────────┤ »
«q_4: ────────────────────────■────────────────■───────────┤ Ry(0.025251) ├─»
«                                              │         ┌─┴──────────────┴┐»
«q_5: ─────────────────────────────────────────■─────────┤ Ry(-1.0324e-05) ├»
«                                                        └─────────────────┘»
«q_6: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«                      
«q_2: ─────────────────
«                      
«q_3: ─────────────────
«       ┌───────────┐  
«q_4: ──┤ Rz(1.011) ├──
«     ┌─┴───────────┴─┐
«q_5: ┤ Rz(-0.046623) ├
«     └───────────────┘
«q_6: ─────────────────
«

In [17]:
#checking circuit

In [18]:
pauli_check_circs = []
phase_y_front = [2.113295958493177e-07,0.0054976714345856634, 0.020630778702533964, 3.40008274438278e-05, 0.02524649597343495, -4.81317308598186e-05]
phase_z_front = [1.3165147216569606, -0.07815775859934079, 0.006091440853078919, 1.528289688855932, -0.009893684371252858, 1.3655876587592497]
phase_y_end = [-7.269945363221252e-06, 0.0055856209568994186, 0.020669506733621706, 2.6996665334365784e-05, 0.02525084679082589, -1.0323843699871446e-05]
phase_z_end = [-0.23806644958543488, -0.21685946937956296, 1.2696203088392648, 0.02265810653600898, 1.010960593326885, -0.04662312860318999]
x_front = [True, False, False, True, False, False]
for i in range(0, 6):
    circuit = QuantumCircuit(7)
    apply_checking_circuit(circuit, [i], [6], side = 'front', phase_y = phase_y_front[i], phase_z = phase_z_front[i], x = x_front[i])
    circuit += orign_circuit
    apply_checking_circuit(circuit, [i], [6], side = 'end', phase_y = phase_y_end[i], phase_z = phase_z_end[i])
    pauli_check_circs.append(circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23955/2850195311.py:10: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += orign_circuit


In [19]:
pauli_check_circs[-1].draw()

┌───┐       ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├───────■───────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘       │       »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├──────────────────────■───────»
      └┬──────────────┤ ├───────────────┤                              »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├──────────────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐               »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├───────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘               »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├─────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘              ┌─────────────┐»
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────■───────┤ Rz(-1.3656) ├»
     └──────┬───┬──────┘  └────────────┘        │       └─────────────┘»
q_6: ───────┤ H ├───────────────────────────────■──────────────────────»
            └───┘                                                      »
«     ┌─────────────────┐  ┌──────────────┐                                   »
«q_0: ┤ Ry(-7.2699e-06) ├──┤ Rz(-0.23807) ├───────────────────────────────────»
«     └─────────────────┘ ┌┴──────────────┤ ┌──────────────┐                  »
«q_1: ─────────■──────────┤ Ry(0.0055856) ├─┤ Rz(-0.21686) ├──────────────────»
«              │          └───────────────┘ ├─────────────┬┘  ┌────────────┐  »
«q_2: ─────────■──────────────────■─────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├──»
«                                 │         └─────────────┘ ┌─┴────────────┴─┐»
«q_3: ────────────────────────────■────────────────■────────┤ Ry(2.6997e-05) ├»
«                                                  │        └────────────────┘»
«q_4: ─────────────────────────────────────────────■────────────────■─────────»
«      ┌────────────────┐┌─────────────────┐ ┌────────────┐         │         »
«q_5: ─┤ Ry(4.8132e-05) ├┤ Ry(-4.8132e-05) ├─┤ Rz(1.3656) ├─────────■─────────»
«      └────────────────┘└─────────────────┘ └────────────┘                   »
«q_6: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«                                                                              »
«q_0: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q_1: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q_2: ─────────────────────────────────────────────────────────────────────────»
«       ┌──────────────┐                                                       »
«q_3: ──┤ Rz(0.022658) ├───────────────────────────────────────────────────────»
«       ├──────────────┤   ┌───────────┐                                       »
«q_4: ──┤ Ry(0.025251) ├───┤ Rz(1.011) ├───────────────────────────────────────»
«     ┌─┴──────────────┴┐┌─┴───────────┴─┐┌──────────────┐┌────────────────┐   »
«q_5: ┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├┤ Rz(0.046623) ├┤ Ry(1.0324e-05) ├─■─»
«     └─────────────────┘└───────────────┘└──────────────┘└────────────────┘ │ »
«q_6: ───────────────────────────────────────────────────────────────────────■─»
«                                                                              »
«                                         
«q_0: ────────────────────────────────────
«                                         
«q_1: ────────────────────────────────────
«                                         
«q_2: ────────────────────────────────────
«                                         
«q_3: ────────────────────────────────────
«                                         
«q_4: ────────────────────────────────────
«     ┌─────────────────┐┌───────────────┐
«q_5: ┤ Ry(-1.0324e-05) ├┤ Rz

In [20]:
pauli_list = read_from_file('BeH2_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [21]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 6, num_qargs = 7)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YXYYXY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_5: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_6: ────────────────
                     
YXYZXZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_3: ────────────────
      ┌────────────┐ 
q_4: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_5: ────────────────
                     
q_6: ────────────────
                     
ZXZYXY
                     
q_0: ────────────────
      ┌────────────┐ 
q_1: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_2: ────────────────
     ┌──────────────┐
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_5: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_6: ────────────────
           

In [22]:
pauli_check_circs[1].draw()

┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├───────────────────────────────■───────────────────────»
            └───┘                                                       »
«                                                            »
«q_0: ─────────────────────────────────────────────────────■─»
«     ┌────────────────┐┌───────────────┐┌───────────────┐ │ »
«q_1: ┤ Ry(-0.0054977) ├┤ Ry(0.0054977) ├┤ Rz(-0.078158) ├─■─»
«     └────────────────┘└───────────────┘└───────────────┘   »
«q_2: ───────────────────────────────────────────────────────»
«                                                            »
«q_3: ───────────────────────────────────────────────────────»
«                                                            »
«q_4: ───────────────────────────────────────────────────────»
«                                                            »
«q_5: ───────────────────────────────────────────────────────»
«                                                            »
«q_6: ───────────────────────────────────────────────────────»
«                                                            »
«     ┌─────────────────┐ ┌──────────────┐                                  »
«q_0: ┤ Ry(-7.2699e-06) ├─┤ Rz(-0.23807) ├──────────────────────────────────»
«     └─────────────────┘┌┴──────────────┤┌──────────────┐ ┌─────────────┐  »
«q_1: ─────────■─────────┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├─┤ Rz(0.21686) ├──»
«              │         └───────────────┘├─────────────┬┘ └┬────────────┤  »
«q_2: ─────────■─────────────────■────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├──»
«                                │        └─────────────┘ ┌─┴────────────┴─┐»
«q_3: ───────────────────────────■───────────────■────────┤ Ry(2.6997e-05) ├»
«                                                │        └────────────────┘»
«q_4: ───────────────────────────────────────────■────────────────■─────────»
«                                                                 │         »
«q_5: ────────────────────────────────────────────────────────────■─────────»
«                                                                           »
«q_6: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«                                                                             
«q_0: ────────────────────────────────────────────────────────────────────────
«      ┌────────────────┐                    ┌───────────────┐┌──────────────┐
«q_1: ─┤ Ry(-0.0055856) ├──────────────────■─┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├
«      └────────────────┘                  │ └───────────────┘└──────────────┘
«q_2: ─────────────────────────────────────┼──────────────────────────────────
«       ┌──────────────┐                   │                                  
«q_3: ──┤ Rz(0.022658) ├───────────────────┼──────────────────────────────────
«       ├──────────────┤   ┌───────────┐   │                                  
«q_4: ──┤ Ry(0.025251) ├───┤ Rz(1.011) ├───┼────────────────────────────────

In [23]:
circuit_list = []
for check_circ in pauli_check_circs:
    for meas in meas_qcs:
        temp_circuit = check_circ.copy()
        temp_circuit += meas
        temp_circuit.measure_active()
        circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23955/1839173154.py:5: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [24]:
len(circuit_list)

204

In [25]:
circuit_list[2].draw()

┌───┐       ┌────────────────┐┌────────────┐   »
      q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─»
            ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘ │ »
      q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────┼─»
            └┬──────────────┤ ├───────────────┤                │ »
      q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼─»
             └────┬───┬─────┘ ├───────────────┴┐┌────────────┐ │ »
      q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├─┼─»
             ┌────┴───┴─────┐ ├────────────────┤└────────────┘ │ »
      q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼─»
           ┌─┴──────────────┴┐└─┬────────────┬─┘               │ »
      q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼─»
           └──────┬───┬──────┘  └────────────┘                 │ »
      q_6: ───────┤ H ├────────────────────────────────────────■─»
                  └───┘                                          »
measure_0: ══════════════════════════════════════════════════════»
                                                                 »
measure_1: ══════════════════════════════════════════════════════»
                                                                 »
measure_2: ══════════════════════════════════════════════════════»
                                                                 »
measure_3: ══════════════════════════════════════════════════════»
                                                                 »
measure_4: ══════════════════════════════════════════════════════»
                                                                 »
measure_5: ══════════════════════════════════════════════════════»
                                                                 »
measure_6: ══════════════════════════════════════════════════════»
                                                                 »
«           ┌─────────────┐┌─────────────────┐┌───┐┌───┐┌────────────────┐»
«      q_0: ┤ Rz(-1.3165) ├┤ Ry(-2.1133e-07) ├┤ X ├┤ X ├┤ Ry(2.1133e-07) ├»
«           └─────────────┘└─────────────────┘└───┘└───┘└────────────────┘»
«      q_1: ──────────────────────────────────────────────────────────────»
«                                                                         »
«      q_2: ──────────────────────────────────────────────────────────────»
«                                                                         »
«      q_3: ──────────────────────────────────────────────────────────────»
«                                                                         »
«      q_4: ──────────────────────────────────────────────────────────────»
«                                                                         »
«      q_5: ──────────────────────────────────────────────────────────────»
«                                                                         »
«      q_6: ──────────────────────────────────────────────────────────────»
«                                                                         »
«measure_0: ══════════════════════════════════════════════════════════════»
«                                                                         »
«measure_1: ══════════════════════════════════════════════════════════════»
«                                                                         »
«measure_2: ══════════════════════════════════════════════════════════════»
«                                                                         »
«measure_3: ══════════════════════════════════════════════════════════════»
«                                                                         »
«measure_4: ══════════════════════════════════════════════════════════════»
«                                                                         »
«measure_5: ══════════════════════════════════════════════════════════════»
«                                                                         »
«measure_6: ═══════════

In [26]:
orign_qc = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2.qasm")

In [27]:
#generate individual measurment circuits that changes the basis
meas_orign_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 6, num_qargs = 6)
    meas_orign_qcs.append(temp_qc)
    print(temp_qc)

YXYYXY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_5: ┤ U(π/2,0,π/2) ├
     └──────────────┘
YXYZXZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_3: ────────────────
      ┌────────────┐ 
q_4: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_5: ────────────────
                     
ZXZYXY
                     
q_0: ────────────────
      ┌────────────┐ 
q_1: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_2: ────────────────
     ┌──────────────┐
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_5: ┤ U(π/2,0,π/2) ├
     └──────────────┘
XXXYXY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_1: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_2:

In [28]:
orign_circuit_list = []
for meas in meas_orign_qcs:
    temp_circuit = orign_qc.copy()
    temp_circuit += meas
    temp_circuit.measure_active()
    orign_circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23955/242718267.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [29]:
len(orign_circuit_list)

34

In [30]:
circuit_list = circuit_list + orign_circuit_list

In [31]:
len(circuit_list)

238

In [32]:
# max_shots = device.configuration().to_dict()['max_shots']
# max_shots

In [33]:
max_shots = 10000

In [34]:
trans_qcs = transpile(circuit_list, device, optimization_level = 3, seed_transpiler = 0)

In [35]:
trans_qcs[0].draw()

»
  ancilla_0 -> 0 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_1 -> 1 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_2 -> 2 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_3 -> 3 ────────────────────────────────────────────────────────────»
                   ┌─────────┐       ┌────┐                                  »
        q_6 -> 4 ──┤ Rz(π/2) ├───────┤ √X ├──────────────────────────────────»
                   └─────────┘       └────┘                                  »
  ancilla_4 -> 5 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_5 -> 6 ────────────────────────────────────────────────────────────»
                 ┌─────────────┐     ┌────┐       ┌──────────┐     ┌────┐    »
        q_0 -> 7 ┤ Rz(-1.5695) ├─────┤ √X ├───────┤ Rz(-π/2) ├─────┤ √X ├────»
                 └─────────────┘     └────┘       └──────────┘     └────┘    »
  ancilla_6 -> 8 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_7 -> 9 ────────────────────────────────────────────────────────────»
                 ┌─────────────┐     ┌────┐     ┌─────────────┐    ┌────┐    »
       q_1 -> 10 ┤ Rz(-1.5695) ├─────┤ √X ├─────┤ Rz(-1.5708) ├────┤ √X ├────»
                 ├─────────────┤     ├────┤     └─┬──────────┬┘    ├────┤    »
       q_5 -> 11 ┤ Rz(-1.5695) ├─────┤ √X ├───────┤ Rz(-π/2) ├─────┤ √X ├────»
                 └┬────────────┤     ├────┤     ┌─┴──────────┴┐    ├────┤    »
       q_2 -> 12 ─┤ Rz(1.5695) ├─────┤ √X ├─────┤ Rz(-1.5708) ├────┤ √X ├────»
                  └───┬────┬───┘ ┌───┴────┴───┐ └────┬────┬───┘┌───┴────┴───┐»
       q_3 -> 13 ─────┤ √X ├─────┤ Rz(3.1416) ├──────┤ √X ├────┤ Rz(-1.527) ├»
                      ├────┤    ┌┴────────────┴┐     ├────┤    └────────────┘»
       q_4 -> 14 ─────┤ √X ├────┤ Rz(0.025246) ├─────┤ √X ├──────────────────»
                      └────┘    └──────────────┘     └────┘                  »
 ancilla_8 -> 15 ────────────────────────────────────────────────────────────»
                                                                             »
 ancilla_9 -> 16 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_10 -> 17 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_11 -> 18 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_12 -> 19 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_13 -> 20 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_14 -> 21 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_15 -> 22 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_16 -> 23 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_17 -> 24 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_18 -> 25 ───────────────────────────────

In [36]:
trans_qcs[-1].draw()

»
  ancilla_0 -> 0 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_1 -> 1 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_2 -> 2 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_3 -> 3 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_4 -> 4 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_5 -> 5 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_6 -> 6 ───────────────────────────────────────────────────────»
                      ┌───┐     ┌─────────────┐                         »
        q_0 -> 7 ─────┤ X ├─────┤ Rz(-1.3165) ├─────────────────────────»
                      └───┘     └─────────────┘                         »
  ancilla_7 -> 8 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_8 -> 9 ───────────────────────────────────────────────────────»
                  ┌────────────┐     ┌────┐    ┌─────────────┐  ┌────┐  »
       q_1 -> 10 ─┤ Rz(1.5695) ├─────┤ √X ├────┤ Rz(-1.5708) ├──┤ √X ├──»
                 ┌┴────────────┤     ├────┤    └─┬──────────┬┘  ├────┤  »
       q_5 -> 11 ┤ Rz(-1.5695) ├─────┤ √X ├──────┤ Rz(-π/2) ├───┤ √X ├──»
                 ├─────────────┤     ├────┤    ┌─┴──────────┴┐  ├────┤  »
       q_2 -> 12 ┤ Rz(-1.5695) ├─────┤ √X ├────┤ Rz(-1.5708) ├──┤ √X ├──»
                 └────┬────┬───┘ ┌───┴────┴───┐└────┬────┬───┘┌─┴────┴─┐»
       q_3 -> 13 ─────┤ √X ├─────┤ Rz(3.1416) ├─────┤ √X ├────┤ Rz(-π) ├»
                  ┌───┴────┴───┐ └───┬────┬───┘ ┌───┴────┴───┐└─┬────┬─┘»
       q_4 -> 14 ─┤ Rz(3.1317) ├─────┤ √X ├─────┤ Rz(-1.596) ├──┤ √X ├──»
                  └────────────┘     └────┘     └────────────┘  └────┘  »
 ancilla_9 -> 15 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_10 -> 16 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_11 -> 17 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_12 -> 18 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_13 -> 19 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_14 -> 20 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_15 -> 21 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_16 -> 22 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_17 -> 23 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_18 -> 24 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_19 -> 25 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_20 -> 26 ───────────────────────────────────────────────────────»
                                                                        »
  

In [37]:
len(trans_qcs)

238

In [38]:
for qc in trans_qcs:
    print(qc.count_ops())

OrderedDict([('rz', 41), ('sx', 31), ('cx', 7), ('measure', 7), ('barrier', 1)])
OrderedDict([('rz', 43), ('sx', 31), ('cx', 7), ('measure', 7), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 42), ('sx', 31), ('cx', 7), ('measure', 7), ('barrier', 1)])
OrderedDict([('rz', 42), ('sx', 31), ('cx', 7), ('measure', 7), ('barrier', 1)])
OrderedDict([('rz', 41), ('sx', 31), ('cx', 7), ('measure', 7), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 42), ('sx', 31), ('cx', 7), ('measure', 7), ('barrier', 1)])
OrderedDict([('rz', 42), ('sx', 31), ('cx', 7), ('measure', 7), ('barrier', 1)])
OrderedDict([('rz', 43), ('sx', 31), ('cx', 7), ('measure', 7), ('barrier', 1)])
OrderedDict([('rz', 41), ('sx', 31), ('cx', 7), ('measure', 7), ('barrier', 1)])
OrderedDict([('rz', 43), ('sx', 31), ('cx', 7), ('measure', 7), ('barrier', 1)])
OrderedDict([('rz', 42), ('sx', 31), ('cx', 7), ('measure', 7), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 43), ('sx', 31), ('cx', 7), ('measure', 7), ('barrier', 1)]

In [39]:
#7,10,12,13,14,11
trans_qcs[0].draw()

»
  ancilla_0 -> 0 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_1 -> 1 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_2 -> 2 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_3 -> 3 ────────────────────────────────────────────────────────────»
                   ┌─────────┐       ┌────┐                                  »
        q_6 -> 4 ──┤ Rz(π/2) ├───────┤ √X ├──────────────────────────────────»
                   └─────────┘       └────┘                                  »
  ancilla_4 -> 5 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_5 -> 6 ────────────────────────────────────────────────────────────»
                 ┌─────────────┐     ┌────┐       ┌──────────┐     ┌────┐    »
        q_0 -> 7 ┤ Rz(-1.5695) ├─────┤ √X ├───────┤ Rz(-π/2) ├─────┤ √X ├────»
                 └─────────────┘     └────┘       └──────────┘     └────┘    »
  ancilla_6 -> 8 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_7 -> 9 ────────────────────────────────────────────────────────────»
                 ┌─────────────┐     ┌────┐     ┌─────────────┐    ┌────┐    »
       q_1 -> 10 ┤ Rz(-1.5695) ├─────┤ √X ├─────┤ Rz(-1.5708) ├────┤ √X ├────»
                 ├─────────────┤     ├────┤     └─┬──────────┬┘    ├────┤    »
       q_5 -> 11 ┤ Rz(-1.5695) ├─────┤ √X ├───────┤ Rz(-π/2) ├─────┤ √X ├────»
                 └┬────────────┤     ├────┤     ┌─┴──────────┴┐    ├────┤    »
       q_2 -> 12 ─┤ Rz(1.5695) ├─────┤ √X ├─────┤ Rz(-1.5708) ├────┤ √X ├────»
                  └───┬────┬───┘ ┌───┴────┴───┐ └────┬────┬───┘┌───┴────┴───┐»
       q_3 -> 13 ─────┤ √X ├─────┤ Rz(3.1416) ├──────┤ √X ├────┤ Rz(-1.527) ├»
                      ├────┤    ┌┴────────────┴┐     ├────┤    └────────────┘»
       q_4 -> 14 ─────┤ √X ├────┤ Rz(0.025246) ├─────┤ √X ├──────────────────»
                      └────┘    └──────────────┘     └────┘                  »
 ancilla_8 -> 15 ────────────────────────────────────────────────────────────»
                                                                             »
 ancilla_9 -> 16 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_10 -> 17 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_11 -> 18 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_12 -> 19 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_13 -> 20 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_14 -> 21 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_15 -> 22 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_16 -> 23 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_17 -> 24 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_18 -> 25 ───────────────────────────────

In [44]:
trans_qcs[-1].draw()

»
  ancilla_0 -> 0 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_1 -> 1 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_2 -> 2 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_3 -> 3 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_4 -> 4 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_5 -> 5 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_6 -> 6 ───────────────────────────────────────────────────────»
                      ┌───┐     ┌─────────────┐                         »
        q_0 -> 7 ─────┤ X ├─────┤ Rz(-1.3165) ├─────────────────────────»
                      └───┘     └─────────────┘                         »
  ancilla_7 -> 8 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_8 -> 9 ───────────────────────────────────────────────────────»
                  ┌────────────┐     ┌────┐    ┌─────────────┐  ┌────┐  »
       q_1 -> 10 ─┤ Rz(1.5695) ├─────┤ √X ├────┤ Rz(-1.5708) ├──┤ √X ├──»
                 ┌┴────────────┤     ├────┤    └─┬──────────┬┘  ├────┤  »
       q_5 -> 11 ┤ Rz(-1.5695) ├─────┤ √X ├──────┤ Rz(-π/2) ├───┤ √X ├──»
                 ├─────────────┤     ├────┤    ┌─┴──────────┴┐  ├────┤  »
       q_2 -> 12 ┤ Rz(-1.5695) ├─────┤ √X ├────┤ Rz(-1.5708) ├──┤ √X ├──»
                 └────┬────┬───┘ ┌───┴────┴───┐└────┬────┬───┘┌─┴────┴─┐»
       q_3 -> 13 ─────┤ √X ├─────┤ Rz(3.1416) ├─────┤ √X ├────┤ Rz(-π) ├»
                  ┌───┴────┴───┐ └───┬────┬───┘ ┌───┴────┴───┐└─┬────┬─┘»
       q_4 -> 14 ─┤ Rz(3.1317) ├─────┤ √X ├─────┤ Rz(-1.596) ├──┤ √X ├──»
                  └────────────┘     └────┘     └────────────┘  └────┘  »
 ancilla_9 -> 15 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_10 -> 16 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_11 -> 17 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_12 -> 18 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_13 -> 19 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_14 -> 20 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_15 -> 21 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_16 -> 22 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_17 -> 23 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_18 -> 24 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_19 -> 25 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_20 -> 26 ───────────────────────────────────────────────────────»
                                                                        »
  

In [58]:
job = execute(circuit_list, backend = device, optimization_level = 3, shots = max_shots, seed_transpiler = 0)
print(job.job_id())

63919226f47ae76ffdf57f75


In [59]:
job_monitor(job)

Job Status: job has successfully run


In [60]:
# job = device.retrieve_job(job_id='638d16954a111e52c2527afb')

In [61]:
result = job.result()

In [62]:
len(result.get_counts())

238

In [66]:
import json
with open('BeH2_auckland_nocut_orign.json', 'w') as f:
    json.dump(result.get_counts(), f)

In [67]:
per_miti_results = []
for i in range(0, 6):
    temp_list = []
    for j in range(0, 34):
        filter_per_miti = norm_dict(filter_results(result.get_counts()[i * 34 + j], [0]))
        temp_list.append(filter_per_miti)
    per_miti_results.append(temp_list)

In [68]:
len(per_miti_results)

6

In [69]:
for j in range(0, 6):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(per_miti_results[j][i], shots = total_counts(per_miti_results[j][i]), Pauli = Pauli_tuple[0])  
    print(final_expect_val)

(-3.3466712073191554+0j)
(-3.3678917464119027+0j)
(-3.2606581088895323+0j)
(-3.339698506617598+0j)
(-3.2650744787368455+0j)
(-3.301480358302736+0j)


In [70]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(result.get_counts()[i + 204], shots = max_shots, Pauli = Pauli_tuple[0]) 

In [71]:
final_expect_val

(-3.3182967192901955+0j)

In [72]:
mapping = mthree.utils.final_measurement_mapping(trans_qcs[-1])
mapping

{7: 0, 10: 1, 12: 2, 13: 3, 14: 4, 11: 5}

In [73]:
mit = mthree.M3Mitigation(device)

In [ ]:
mit.cals_from_system(list(mapping))

In [ ]:
mit.cals_to_file(cals_file = 'mit_data_7_10_12_13_14_11')